In [3]:
import os
import pandas as pd
import streamlit as st

In [2]:
CSV_PATH = "C:/code/agency_dash/agengy/contas_config.csv"

In [4]:
def carregar_dados():
    df = pd.read_csv("relatorio_analytics_30dias_tratado.csv", sep=";")
    df.columns = df.columns.str.strip()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    return df

df = carregar_dados()

In [ ]:
# -----------------------
# 🔹 Carrega os dados do CSV
# -----------------------
if os.path.exists(CSV_PATH):
    df = pd.read_csv(CSV_PATH, sep=";")

# Tenta localizar a conta no arquivo
row = df[df["conta"] == conta]

# if row.empty:
#     st.error("Conta não encontrada no arquivo.")
#     return

# -----------------------
# 🔹 Inicializa os dados da sessão
# -----------------------
if "edit_data" not in st.session_state:
    # Lê informações existentes
    status = row["status"].iloc[0] if "status" in row else "Ativo"
    meta = float(row["meta"].iloc[0]) if "meta" in row else 0.0

    link1 = row["link1"].iloc[0] if "link1" in row else ""
    link2 = row["link2"].iloc[0] if "link2" in row else ""
    link3 = row["link3"].iloc[0] if "link3" in row else ""
    link4 = row["link4"].iloc[0] if "link4" in row else ""
    link5 = row["link5"].iloc[0] if "link5" in row else ""
    link6 = row["link6"].iloc[0] if "link6" in row else ""